# Drowiness Detection System

This project shows the **open eyes / close eyes ML detector** and was developed using the Python:

   * OpenCV - face and eye detection
   * Tensorflow - ML backend used by Keras
   * Keras - build classification model
   * Pygame - to play alarm sound

**Step 1** Take image as input from a camera<br>

**Step 2** Detect the face in the image and create the Region Of Interest(ROI)<br>

**Step 3** Detect the left and right in the ROI and feed it to the classifier<br>

**Step 4** Classifier will categorize whether eyes are open or closed<br>

**Step 5** Calculate score to check whether the person is drowsy<br>

In [ ]:
!pip install keras

In [ ]:
!pip install pygame

In [1]:
import cv2 as cv
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

pygame 2.1.0 (SDL 2.0.16, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
mixer.init()
sound = mixer.Sound('alarm.wav')

In [3]:
# whole face = f
face = cv.CascadeClassifier('/Users/adunifekizitookoye/opt/anaconda3/lib/python3.8/site-packages/cv2/data/haarcascade_frontalface_alt.xml')
#left eye = l_e
l_e = cv.CascadeClassifier('/Users/adunifekizitookoye/opt/anaconda3/lib/python3.8/site-packages/cv2/data/haarcascade_lefteye_2splits.xml')
#right eye = r_e
r_e = cv.CascadeClassifier('/Users/adunifekizitookoye/opt/anaconda3/lib/python3.8/site-packages/cv2/data/haarcascade_righteye_2splits.xml') 

In [4]:
lbl = ['Close', 'Open']

In [5]:
model = load_model('models/cnncat2.h5')
path = os.getcwd() #gets current working directory from operating system

cap = cv.VideoCapture(0)
font = cv.FONT_HERSHEY_COMPLEX_SMALL

count = 0
score = 0
thicc = 2
rpred =[99]
lpred =[99]

2021-11-19 03:04:18.839909: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
while True:
    ret, frame = cap.read()
    
    height, width = frame.shape[:2]
     
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    leye = l_e.detectMultiScale(gray)
    reye = r_e.detectMultiScale(gray)
    
    cv.rectangle(frame, (0, height-50), (200,height), (0,0,0), thickness=cv.FILLED)
    
    for (x,y,w,h) in faces:
        cv.rectangle(frame,(x,y), (x+w, y+h), (100,100,100), 1)
    
    # right eye
    for (x,y,w,h) in reye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv.cvtColor(r_eye,cv.COLOR_BGR2GRAY)
        r_eye = cv.resize(r_eye,(24,24))
        r_eye= r_eye/255
        r_eye=  r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = model.predict(r_eye)
        if(rpred[0]==1):
            lbl='Open' 
        if(rpred[0]==0):
            lbl='Closed'
        break
        
    # left eye
    for (x,y,w,h) in leye:
        l_eye = frame[y:y+h, x:x+w]
        count = count + 1
        l_eye = cv.cvtColor(l_eye, cv.COLOR_BGR2GRAY)
        
        l_eye = cv.resize(l_eye, (24, 24))
        l_eye = l_eye/255
        l_eye = l_eye.reshape(24, 24, -1)
        l_eye = np.expand_dims(l_eye, axis=0)
        lpred = model.predict(l_eye)
        if(lpred[0]==1):
            lbl = 'Open'
        if(lpred[0]==0):
            lbl = 'Closed'
        break
        
        
    if(rpred[0]==0 and lpred[0]==0):
        score = score + 1
        cv.putText(frame, "Closed", (10, height-20), font, 1, (255, 255, 255), 1, cv.LINE_AA)
    #if(rpred[0]==1 or lpred[0]==0)
    else:
        score = score - 1
        cv.putText(frame, "Open", (10, height-20), font, 1, (255, 255, 255), 1, cv.LINE_AA)
        
        
    if(score < 0):
        score = 0
    cv.putText(frame, 'Score: ' +str(score), (100, height-20), font, 1, (255, 255, 255), 1, cv.LINE_AA)
    if(score < 15):
        #cv.imwrite(os.path.join(path, 'image.jpg'), frame)
        try:
            sound.play()
        except:
            pass
        if(thicc < 16):
            thicc = thicc + 2
        else:
            thicc = thicc - 2
            if(thicc < 2):
                thicc = 2
        cv.rectangle(frame, (0,0), (width, height), (0, 0, 255), thicc)
    cv.imshow('frame', frame)
    
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()        

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
!pip uninstall pygame

In [ ]:
!pip uninstall keras

Resources:
* data-flair:[data-flair.training](https://data-flair.training/blogs/python-project-driver-drowsiness-detection-system/)